In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
sns.set()
import pandas as pd
import gc
gc.collect()
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
from tqdm import tqdm

from pyhive import presto
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
requests.packages.urllib3.disable_warnings()
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import gc
gc.collect()
gc.collect()

tqdm.pandas()

/home/ronak/anaconda3/envs/customer_v1/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [14]:
order_timer_rendered_file='orderTimeRendered_20200218.csv'

order_timer_rendered=pd.read_csv(order_timer_rendered_file)

order_timer_rendered.shape


order_timer_rendered.duplicated().sum()

order_timer_rendered.head(2)

rider_rejected_file='riderRejected_20200218.csv'

rider_rejected=pd.read_csv(rider_rejected_file)

rider_rejected.shape
rider_rejected.head()

rider_busy_file='rider_busy_20200218.csv'

rider_busy=pd.read_csv(rider_busy_file)

rider_busy.shape
rider_busy.head()


rider_otw_file='rider_otw_20200218.csv'

rider_otw=pd.read_csv(rider_otw_file)

rider_otw.shape
rider_otw.head()

rider_cust_canc_file='rider_cust_canc_20200218.csv'

rider_cust_canc=pd.read_csv(rider_cust_canc_file)

rider_cust_canc.shape
rider_cust_canc.head()

In [68]:

order_timer_rendered_file='orderTimeRendered_20200218.csv'
lhs=pd.read_csv(order_timer_rendered_file,parse_dates=['time'])
lhs.drop_duplicates(inplace=True)

In [30]:


rhs_files=[
    'riderRejected_20200218.csv',

'rider_busy_20200218.csv',

'rider_otw_20200218.csv',

'rider_cust_canc_20200218.csv',

    
]


In [56]:
dfs=[]
for file in rhs_files:
    df=pd.read_csv(file,parse_dates=['time'])
    dfs.append(df)


In [57]:
all_rhs=pd.concat(dfs)

In [58]:
all_rhs.drop_duplicates(inplace=True)

In [59]:
all_rhs.shape

(1072056, 6)

In [60]:
all_rhs.sort_index(inplace=True)

In [61]:
all_rhs.head()

,eventName,time,order_id,captain_id,app_version,cancel_reason
0,CaptainOrderStatus-riderRejected,2020-02-18 00:00:33,5e4adbbc865d9017b53de9c7,5c6becaa21163e7a7c540484,3.10.5,NaN
0,CaptainOrderStatus-onTheWay,2020-02-18 00:00:02,5e4ad582865d9017b53de2dd,5e2b0f65257046721c75b267,3.10.5,NaN
0,CaptainOrderStatus-riderBusy,2020-02-18 00:00:32,5e4adb912806995062b9f6e6,5bb47306b7078153f3add9e9,3.10.5,NaN
0,CaptainOrderStatus-customerCancelled,2020-02-18 00:01:18,5e4adbe9865d9017b53de9d6,"5d5c0cc2a24c96105e1bc469,5d5c1bbca24c96105e1bdcff",3.10.5,order cancelled before rider accepted
1,CaptainOrderStatus-riderRejected,2020-02-18 00:01:16,5e4adb0a9a773437e4b7a85e,"5c35185f4a267149c769e3e7,5d3d39f09996733edb0519f6",3.10.5,NaN


#### Identify customer cancelled immediately after orderrendered

In [275]:
lhs.shape[0]

1201116

In [76]:
lhs.shape[0]+all_rhs.shape[0]

2273172

In [84]:
lhs.head(2)
all_rhs.head(2)

,eventName,time,captain_id,order_id,app_version
0,orderTimerRendered,2020-02-18 10:30:07,5dca7eba010c5a4601fbfb7f,5e4b6f2b9a773437e4b86e94,3.10.5
1,orderTimerRendered,2020-02-18 10:30:09,5d2823c406ee2c37f1f3937e,5e4b6f2c9a773437e4b86e97,3.10.5


,eventName,time,order_id,captain_id,app_version,cancel_reason
0,CaptainOrderStatus-riderRejected,2020-02-18 00:00:33,5e4adbbc865d9017b53de9c7,5c6becaa21163e7a7c540484,3.10.5,NaN
0,CaptainOrderStatus-onTheWay,2020-02-18 00:00:02,5e4ad582865d9017b53de2dd,5e2b0f65257046721c75b267,3.10.5,NaN


In [97]:
all_events=pd.concat([lhs,all_rhs])

In [98]:
all_events.set_index('time',inplace=True)
all_events.sort_index(inplace=True)

In [100]:
all_events.head(2)
all_events.tail(2)

,eventName,captain_id,order_id,app_version,cancel_reason
time,,,,,
2020-02-18,CaptainOrderStatus-onTheWay,5c822d8e8c352421eadf6ae6,5e4adb4f865d9017b53de988,3.8.8,NaN
2020-02-18,CaptainOrderStatus-riderBusy,5df520d6d01edf2b243f42c3,5e4adb139a773437e4b7a86d,3.10.5,NaN


,eventName,captain_id,order_id,app_version,cancel_reason
time,,,,,
2020-02-18 23:59:59,CaptainOrderStatus-onTheWay,5b3f3d6e73d14c5daa033913,5e4c2d00865d9017b540f5da,3.10.5,NaN
2020-02-18 23:59:59,CaptainOrderStatus-onTheWay,5e33a07e4cb0d041d02dd6a4,5e4c2d1d865d9017b540f5ea,3.10.5,NaN


In [101]:
def get_first_n_elements(x):
    return x.head(3)

In [103]:
order_seq=all_events.head(5000).groupby('order_id').apply(get_first_n_elements)

In [106]:
order_seq.eventName.value_counts()

orderTimerRendered                      1816
CaptainOrderStatus-onTheWay              674
CaptainOrderStatus-riderBusy             366
CaptainOrderStatus-customerCancelled     299
CaptainOrderStatus-riderRejected         100
Name: eventName, dtype: int64

In [111]:
order_seq.loc['5e4acc1e9a773437e4b79384']

,eventName,captain_id,order_id,app_version,cancel_reason
time,,,,,
2020-02-18 00:04:40,CaptainOrderStatus-customerCancelled,"5c559bd403a62d32bb1a4dbc,5c1b97063f491d0a6e5fcfae",5e4acc1e9a773437e4b79384,3.10.5,FE - Accident / Rain / Strike / vehicle issues


In [310]:
all_events[all_events['eventName']=='CaptainOrderStatus-customerCancelled']
all_events_d=all_events.reset_index()

,eventName,captain_id,order_id,app_version,cancel_reason
time,,,,,
2020-02-18 00:00:01,CaptainOrderStatus-customerCancelled,"5d5e3d3095e84053d76aa640,5d5edffcd0286d106d5e6ded",5e4adba62806995062b9f6fb,3.10.5,order cancelled before rider accepted
2020-02-18 00:00:05,CaptainOrderStatus-customerCancelled,"5c6baf62ba8dd579bec56e48,5c811bc45e042733c9c513c1",5e4adb53865d9017b53de98f,3.10.5,Captain is unreachable / switched off
2020-02-18 00:00:07,CaptainOrderStatus-customerCancelled,5e3bde7e4968b62372432288,5e4adba99a773437e4b7a8ee,4.9.8,order cancelled before rider accepted
2020-02-18 00:00:10,CaptainOrderStatus-customerCancelled,"5c6baf62ba8dd579bec56e48,5c811bc45e042733c9c513c1",5e4adb53865d9017b53de98f,3.10.5,order cancelled before rider accepted
2020-02-18 00:00:10,CaptainOrderStatus-customerCancelled,5e19cbb8e556205f524c2f5e,5e4ada832806995062b9f5fb,3.10.5,Captain denied duty
...,...,...,...,...,...
2020-02-18 23:59:55,CaptainOrderStatus-customerCancelled,5e23de73430121028f98e6fe,5e4c2d18865d9017b540f5e6,3.10.5,order cancelled before rider accepted
2020-02-18 23:59:55,CaptainOrderStatus-customerCancelled,5ce98fc025ee3218d4e03f27,5e4c2cd0865d9017b540f5b1,3.10.5,I expected a shorter wait time
2020-02-18 23:59:56,CaptainOrderStatus-customerCancelled,5d09d565c4021c473c918045,5e4c2c86865d9017b540f573,3.10.5,Changed my mind


In [311]:
all_events_d.shape

(2273172, 6)

In [121]:
cancelled_indices=all_events_d[all_events_d['eventName']=='CaptainOrderStatus-customerCancelled'].index

In [124]:
cancelled_indices


Int64Index([     16,      38,      57,      71,      73,      76,      80,
                 81,      85,      90,
            ...
            2273077, 2273080, 2273098, 2273100, 2273128, 2273133, 2273134,
            2273139, 2273146, 2273158],
           dtype='int64', length=193664)

In [126]:
all_events_d.iloc[prev_indices]

,time,eventName,captain_id,order_id,app_version,cancel_reason
15,2020-02-18 00:00:01,orderTimerRendered,5a562c4da817452d0d97c7c9,5e4adb9f865d9017b53de9bb,3.10.5,NaN
37,2020-02-18 00:00:05,orderTimerRendered,5bd43b8e85a5a909ddbed1ce,5e4adba49a773437e4b7a8eb,3.10.5,NaN
56,2020-02-18 00:00:07,orderTimerRendered,"5c85174b5e042733c9c6f67b,5d4d8f10ff23f474dd004bf8",5e4adba22806995062b9f6f7,3.10.5,NaN
70,2020-02-18 00:00:10,CaptainOrderStatus-riderBusy,5a8d44ac49bc7e35bf5149f8,5e4adb612806995062b9f6c8,3.10.5,NaN
72,2020-02-18 00:00:10,orderTimerRendered,5c1689cce3edca287ee2e3d5,5e4adbaf865d9017b53de9c2,3.10.5,NaN
...,...,...,...,...,...,...
2273132,2020-02-18 23:59:55,orderTimerRendered,5ca33bc738f05017d57885ab,5e4c2d20865d9017b540f5ed,3.10.5,NaN
2273133,2020-02-18 23:59:55,CaptainOrderStatus-customerCancelled,5e23de73430121028f98e6fe,5e4c2d18865d9017b540f5e6,3.10.5,order cancelled before rider accepted
2273138,2020-02-18 23:59:56,CaptainOrderStatus-riderBusy,5cefc531377155163c0afbd2,5e4c2d0a2806995062bd04eb,3.10.5,NaN
2273145,2020-02-18 23:59:57,CaptainOrderStatus-onTheWay,5d74ebd5d0286d106d7fa958,5e4c27f09a773437e4baafeb,3.10.5,NaN


In [127]:
all_events_d[all_events_d['eventName']=='CaptainOrderStatus-customerCancelled'].head(2)
all_events_d.iloc[10:20]

,time,eventName,captain_id,order_id,app_version,cancel_reason
16,2020-02-18 00:00:01,CaptainOrderStatus-customerCancelled,"5d5e3d3095e84053d76aa640,5d5edffcd0286d106d5e6ded",5e4adba62806995062b9f6fb,3.10.5,order cancelled before rider accepted
38,2020-02-18 00:00:05,CaptainOrderStatus-customerCancelled,"5c6baf62ba8dd579bec56e48,5c811bc45e042733c9c513c1",5e4adb53865d9017b53de98f,3.10.5,Captain is unreachable / switched off


,time,eventName,captain_id,order_id,app_version,cancel_reason
10,2020-02-18 00:00:01,CaptainOrderStatus-onTheWay,5d1635e8668011467e263ac6,5e4adb48865d9017b53de985,3.10.5,NaN
11,2020-02-18 00:00:01,orderTimerRendered,5ce7cf7725ee3218d4dcbd91,5e4adba19a773437e4b7a8e8,3.10.5,NaN
12,2020-02-18 00:00:01,orderTimerRendered,5dd01dde0856f1461832a37b,5e4adb922806995062b9f6e7,3.10.5,NaN
13,2020-02-18 00:00:01,CaptainOrderStatus-onTheWay,5b6bf9056a6dd54c6dadb434,5e4adba29a773437e4b7a8ea,3.9.4,NaN
14,2020-02-18 00:00:01,orderTimerRendered,5dd6c37661ebec69bf939662,5e4ad9762806995062b9f4ff,3.10.5,NaN
15,2020-02-18 00:00:01,orderTimerRendered,5a562c4da817452d0d97c7c9,5e4adb9f865d9017b53de9bb,3.10.5,NaN
16,2020-02-18 00:00:01,CaptainOrderStatus-customerCancelled,"5d5e3d3095e84053d76aa640,5d5edffcd0286d106d5e6ded",5e4adba62806995062b9f6fb,3.10.5,order cancelled before rider accepted
17,2020-02-18 00:00:01,orderTimerRendered,"5dcaf18e010c5a4601fd405b,5e4a49c0b1b0c465f2e36a99",5e4adba52806995062b9f6f8,3.10.5,NaN
18,2020-02-18 00:00:02,CaptainOrderStatus-onTheWay,5e11b122715b3f22b966f551,5e4adba19a773437e4b7a8e7,3.10.5,NaN
19,2020-02-18 00:00:02,orderTimerRendered,5d2d28a06ed7674e611373c4,5e4adba22806995062b9f6f6,3.10.5,NaN


In [129]:
gc.collect()

38

In [305]:
all_events_d.sort_values('time',inplace=True)

In [307]:
all_events_d.head(2) 
all_events_d.tail(2)

,time,eventName,captain_id,order_id,app_version,cancel_reason
0,2020-02-18,CaptainOrderStatus-onTheWay,5c822d8e8c352421eadf6ae6,5e4adb4f865d9017b53de988,3.8.8,NaN
1,2020-02-18,CaptainOrderStatus-riderBusy,5df520d6d01edf2b243f42c3,5e4adb139a773437e4b7a86d,3.10.5,NaN


,time,eventName,captain_id,order_id,app_version,cancel_reason
2273166,2020-02-18 23:59:59,CaptainOrderStatus-onTheWay,5b1d58e79914c53952430f80,5e4c2be89a773437e4bab439,3.10.5,NaN
2273171,2020-02-18 23:59:59,CaptainOrderStatus-onTheWay,5e33a07e4cb0d041d02dd6a4,5e4c2d1d865d9017b540f5ea,3.10.5,NaN


In [312]:
all_events_d=all_events_d.groupby(['order_id'],group_keys=False).progress_apply(lambda x: x.sort_values('time'))

100%|██████████| 595276/595276 [09:32<00:00, 1040.03it/s]


In [313]:
sorted_events=all_events_d.groupby(['order_id','captain_id']).progress_apply(lambda x: x.sort_values('time'))

100%|██████████| 949697/949697 [15:45<00:00, 1003.94it/s]


In [314]:
sorted_events.head(2)

,,,time,eventName,captain_id,order_id,app_version,cancel_reason
order_id,captain_id,,,,,,,
5e39137351b5a35da32437c3,5d6726d05176ff1173707034,1577691,2020-02-18 18:50:10,orderTimerRendered,5d6726d05176ff1173707034,5e39137351b5a35da32437c3,3.10.4,NaN
5e3aad806c2b60097fd384d0,5bd7c083a0b5cd48cd97845a,1215859,2020-02-18 16:13:31,orderTimerRendered,5bd7c083a0b5cd48cd97845a,5e3aad806c2b60097fd384d0,3.10.5,NaN


In [315]:
sorted_events.to_csv('sorted_events.csv')

#### analysis_of_customer_cancelled

In [316]:
sorted_events.head(50)

,,,time,eventName,captain_id,order_id,app_version,cancel_reason
order_id,captain_id,,,,,,,
5e39137351b5a35da32437c3,5d6726d05176ff1173707034,1577691,2020-02-18 18:50:10,orderTimerRendered,5d6726d05176ff1173707034,5e39137351b5a35da32437c3,3.10.4,NaN
5e3aad806c2b60097fd384d0,5bd7c083a0b5cd48cd97845a,1215859,2020-02-18 16:13:31,orderTimerRendered,5bd7c083a0b5cd48cd97845a,5e3aad806c2b60097fd384d0,3.10.5,NaN
5e44fe5ef1a3072ae0e024a9,5d373cdb8f76d565356ab533,32644,2020-02-18 01:34:32,CaptainOrderStatus-customerCancelled,5d373cdb8f76d565356ab533,5e44fe5ef1a3072ae0e024a9,3.10.5,Food not delivered
5e45ab09865d9017b5328bb2,"5c7416d95e042733c9be335d,5c70948ede141933060f9acb",2248441,2020-02-18 23:32:16,orderTimerRendered,"5c7416d95e042733c9be335d,5c70948ede141933060f9acb",5e45ab09865d9017b5328bb2,3.10.5,NaN
5e46880c9a773437e4ae1a8d,5cd04a003a47bc2391fd3b10,1481540,2020-02-18 18:19:04,orderTimerRendered,5cd04a003a47bc2391fd3b10,5e46880c9a773437e4ae1a8d,3.10.5,NaN
5e476db62806995062b24615,5d3c064bbc80ee0ae84c1763,1587425,2020-02-18 18:53:18,orderTimerRendered,5d3c064bbc80ee0ae84c1763,5e476db62806995062b24615,3.10.5,NaN
5e47b94f2806995062b32a87,5d7251069f130346e2c92323,1334909,2020-02-18 17:18:05,CaptainOrderStatus-customerCancelled,5d7251069f130346e2c92323,5e47b94f2806995062b32a87,3.10.5,unable to reach captain
5e487bc9865d9017b538cb9d,5c444b954a267149c7754c8b,2271465,2020-02-18 23:57:35,orderTimerRendered,5c444b954a267149c7754c8b,5e487bc9865d9017b538cb9d,3.10.5,NaN
5e48e49e865d9017b5394b34,"5c7115bc5e042733c9bca5fe,5d1d7cbe11e17032dceaa3b7",1567248,2020-02-18 18:46:54,orderTimerRendered,"5c7115bc5e042733c9bca5fe,5d1d7cbe11e17032dceaa3b7",5e48e49e865d9017b5394b34,3.10.5,NaN


#### Get riderBusy events with no rendered event for a captain

In [317]:
sorted_events_reset=sorted_events.reset_index(drop=True)
sorted_events_reset.head(30)

,time,eventName,captain_id,order_id,app_version,cancel_reason
0,2020-02-18 18:50:10,orderTimerRendered,5d6726d05176ff1173707034,5e39137351b5a35da32437c3,3.10.4,NaN
1,2020-02-18 16:13:31,orderTimerRendered,5bd7c083a0b5cd48cd97845a,5e3aad806c2b60097fd384d0,3.10.5,NaN
2,2020-02-18 01:34:32,CaptainOrderStatus-customerCancelled,5d373cdb8f76d565356ab533,5e44fe5ef1a3072ae0e024a9,3.10.5,Food not delivered
3,2020-02-18 23:32:16,orderTimerRendered,"5c7416d95e042733c9be335d,5c70948ede141933060f9acb",5e45ab09865d9017b5328bb2,3.10.5,NaN
4,2020-02-18 18:19:04,orderTimerRendered,5cd04a003a47bc2391fd3b10,5e46880c9a773437e4ae1a8d,3.10.5,NaN
5,2020-02-18 18:53:18,orderTimerRendered,5d3c064bbc80ee0ae84c1763,5e476db62806995062b24615,3.10.5,NaN
6,2020-02-18 17:18:05,CaptainOrderStatus-customerCancelled,5d7251069f130346e2c92323,5e47b94f2806995062b32a87,3.10.5,unable to reach captain
7,2020-02-18 23:57:35,orderTimerRendered,5c444b954a267149c7754c8b,5e487bc9865d9017b538cb9d,3.10.5,NaN
8,2020-02-18 18:46:54,orderTimerRendered,"5c7115bc5e042733c9bca5fe,5d1d7cbe11e17032dceaa3b7",5e48e49e865d9017b5394b34,3.10.5,NaN
9,2020-02-18 23:54:29,orderTimerRendered,5d1d96e493686945e711bb94,5e48f9a72806995062b581e9,3.10.5,NaN


In [318]:
def get_rider_busy_and_rendered(x):
    return x[x['eventName'].isin(['orderTimerRendered','CaptainOrderStatus-riderBusy'])]
def filter_render_after_rider_busy(test):
    test.reset_index(drop=True,inplace=True)

    rider_busy_indices=test[test['eventName']=='CaptainOrderStatus-riderBusy'].index
    def prev_event_ordertimer(test,prev_index):
        try:
            return test.iloc[prev_index]['eventName']=='orderTimerRendered'
        except:
            return False
    result=[(prev_event_ordertimer(test,prev_index)) for prev_index in  rider_busy_indices-1]
    l=list(zip(result,rider_busy_indices))
    l
    r2=list(filter(lambda x: x[0]==True, l))
    succes_index=[x[1] for x in r2]
    return test.iloc[succes_index]
slice_frame=sorted_events_reset
rb_and_rendered=slice_frame.groupby(['order_id','captain_id'],group_keys=False).progress_apply(get_rider_busy_and_rendered)


100%|██████████| 949697/949697 [19:11<00:00, 824.65it/s] 


In [319]:
valid_rider_b=rb_and_rendered.groupby(['order_id','captain_id'],group_keys=False).progress_apply(filter_render_after_rider_busy).reset_index(drop=True)

 80%|████████  | 697771/871931 [22:48<05:49, 498.59it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [320]:
valid_rider_b.head()

,time,eventName,captain_id,order_id,app_version,cancel_reason
0,2020-02-18 01:27:32,CaptainOrderStatus-riderBusy,5cbee92b3d65ca5e2560ac41,5e499c1d865d9017b53ac1ae,3.10.5,NaN
1,2020-02-18 05:29:55,CaptainOrderStatus-riderBusy,5d12261f4253186bbaff6174,5e49a3159a773437e4b48189,3.10.2,NaN
2,2020-02-18 05:30:25,CaptainOrderStatus-riderBusy,5d12261f4253186bbaff6174,5e49a3159a773437e4b48189,3.10.2,NaN
3,2020-02-18 05:34:26,CaptainOrderStatus-riderBusy,5e13e93639136622c08063f1,5e49a3159a773437e4b48189,3.10.5,NaN
4,2020-02-18 03:55:30,CaptainOrderStatus-riderBusy,5cf83707ca6e29211173b345,5e4a54bf865d9017b53c44c8,3.10.5,NaN


In [321]:
valid_rider_b.loc[:,'prev_event_order_timer']=True

In [322]:
cols_valid_rider=['time','eventName','captain_id','order_id','prev_event_order_timer']

In [323]:
result=pd.merge(rb_and_rendered,valid_rider_b[cols_valid_rider],how='left',on=['time','eventName','captain_id','order_id'])

In [324]:
result.head(2)
total_number_of_Rider_busy_events=result[result['eventName']=='CaptainOrderStatus-riderBusy'].shape[0]
prev_event_not_order_Timer=result[result['eventName']=='CaptainOrderStatus-riderBusy']['prev_event_order_timer'].isna().sum()
prev_event_order_Timer=(result[result['eventName']=='CaptainOrderStatus-riderBusy']['prev_event_order_timer']==True).sum()

,time,eventName,captain_id,order_id,app_version,cancel_reason,prev_event_order_timer
0,2020-02-18 18:50:10,orderTimerRendered,5d6726d05176ff1173707034,5e39137351b5a35da32437c3,3.10.4,NaN,NaN
1,2020-02-18 16:13:31,orderTimerRendered,5bd7c083a0b5cd48cd97845a,5e3aad806c2b60097fd384d0,3.10.5,NaN,NaN


In [325]:
'total_number_of_Rider_busy_events ='+ str(total_number_of_Rider_busy_events)
'prev_event_not_order_Timer= '+str(prev_event_not_order_Timer)
'prev_event_order_Timer='+str(prev_event_order_Timer)

'total_number_of_Rider_busy_events =228634'

'prev_event_not_order_Timer= 36235'

'prev_event_order_Timer=192399'

In [273]:
# 'total_number_of_Rider_busy_events ='+ str(total_number_of_Rider_busy_events)
# 'prev_event_not_order_Timer= '+str(prev_event_not_order_Timer)
# 'prev_event_order_Timer='+str(prev_event_order_Timer)

'total_number_of_Rider_busy_events =228634'

'prev_event_not_order_Timer= 36232'

'prev_event_order_Timer=192402'

In [274]:
result.to_csv('marked_prev_event_ordertimer_Renderd.csv')

#### Events before prev not order timer

In [329]:
result=result.groupby('order_id',group_keys=False).progress_apply(lambda x: x.sort_values('time'))

 71%|███████   | 406020/573344 [05:10<02:02, 1365.00it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [330]:
s=result[result['eventName']=='CaptainOrderStatus-riderBusy']['prev_event_order_timer'].isna()
indices=s[s].index
prev_curr=[*indices,*indices-1]
prev_curr.sort()
result.iloc[prev_curr]
result.iloc[indices-1]['eventName'].value_counts()
# result[]

,time,eventName,captain_id,order_id,app_version,cancel_reason,prev_event_order_timer
13,2020-02-18 01:26:39,orderTimerRendered,5b7bdcfb841e3005f30e267b,5e499c1d865d9017b53ac1ae,3.10.5,NaN,NaN
14,2020-02-18 01:27:04,CaptainOrderStatus-riderBusy,5cbee92b3d65ca5e2560ac41,5e499c1d865d9017b53ac1ae,3.10.5,NaN,NaN
34,2020-02-18 00:08:49,orderTimerRendered,5db51a01f53c58214265a677,5e49bfbc865d9017b53ac859,3.10.5,NaN,NaN
38,2020-02-18 06:53:24,orderTimerRendered,"5ddba72e6ffaa931412b2dee,5decdbcefa8847337e1c2158",5e49f2a0865d9017b53ae57e,3.10.5,NaN,NaN
45,2020-02-18 01:00:58,orderTimerRendered,5cf23eeec0b26018f5d51784,5e4a17392806995062b77b7d,3.10.5,NaN,NaN
...,...,...,...,...,...,...,...
1410341,2020-02-18 23:59:14,orderTimerRendered,59e61aeb41f9a3a136c042c4,5e4c2cd69a773437e4bab535,3.10.5,NaN,NaN
1410525,2020-02-18 23:59:23,orderTimerRendered,5de278f0d472d62e9459600c,5e4c2d009a773437e4bab553,3.10.2,NaN,NaN
1410526,2020-02-18 23:59:29,orderTimerRendered,5de278f0d472d62e9459600c,5e4c2d009a773437e4bab553,3.10.2,NaN,NaN
1410558,2020-02-18 23:59:32,orderTimerRendered,5e023ce618576832ff7b7ac8,5e4c2d0a9a773437e4bab559,3.10.5,NaN,NaN


orderTimerRendered              24344
CaptainOrderStatus-riderBusy    11891
Name: eventName, dtype: int64

In [327]:
res_data=result.iloc[indices]
rider_busy_indices=res_data[res_data['eventName']=='CaptainOrderStatus-riderBusy'].index
prev_curr_rb=[*rider_busy_indices,*rider_busy_indices-1]
prev_curr_rb.sort()
result.iloc[prev_curr_rb].head(20)



,time,eventName,captain_id,order_id,app_version,cancel_reason,prev_event_order_timer
13,2020-02-18 01:26:39,orderTimerRendered,5b7bdcfb841e3005f30e267b,5e499c1d865d9017b53ac1ae,3.10.5,NaN,NaN
14,2020-02-18 01:27:04,CaptainOrderStatus-riderBusy,5cbee92b3d65ca5e2560ac41,5e499c1d865d9017b53ac1ae,3.10.5,NaN,NaN
34,2020-02-18 00:08:49,orderTimerRendered,5db51a01f53c58214265a677,5e49bfbc865d9017b53ac859,3.10.5,NaN,NaN
35,2020-02-18 06:53:47,CaptainOrderStatus-riderBusy,5c78ffe75e042733c9c11128,5e49f2a0865d9017b53ae57e,3.9.2,NaN,NaN
46,2020-02-18 01:01:02,orderTimerRendered,5cf23eeec0b26018f5d51784,5e4a17392806995062b77b7d,3.10.5,NaN,NaN
47,2020-02-18 01:00:54,CaptainOrderStatus-riderBusy,5d92f20861bf5a14d08f6634,5e4a17392806995062b77b7d,3.10.5,NaN,NaN
47,2020-02-18 01:00:54,CaptainOrderStatus-riderBusy,5d92f20861bf5a14d08f6634,5e4a17392806995062b77b7d,3.10.5,NaN,NaN
48,2020-02-18 01:01:30,CaptainOrderStatus-riderBusy,5d92f20861bf5a14d08f6634,5e4a17392806995062b77b7d,3.10.5,NaN,NaN
71,2020-02-18 00:07:55,orderTimerRendered,5df100cb1471f12b2a90694c,5e4a755d9a773437e4b65755,3.10.5,NaN,NaN
72,2020-02-18 00:07:10,CaptainOrderStatus-riderBusy,5e257a48430121028f9b1a88,5e4a755d9a773437e4b65755,NaN,NaN,NaN


In [298]:
result.iloc[indices-1]

,time,eventName,captain_id,order_id,app_version,cancel_reason,prev_event_order_timer
13,2020-02-18 01:26:39,orderTimerRendered,5b7bdcfb841e3005f30e267b,5e499c1d865d9017b53ac1ae,3.10.5,NaN,NaN
34,2020-02-18 00:08:49,orderTimerRendered,5db51a01f53c58214265a677,5e49bfbc865d9017b53ac859,3.10.5,NaN,NaN
46,2020-02-18 01:01:02,orderTimerRendered,5cf23eeec0b26018f5d51784,5e4a17392806995062b77b7d,3.10.5,NaN,NaN
47,2020-02-18 01:00:54,CaptainOrderStatus-riderBusy,5d92f20861bf5a14d08f6634,5e4a17392806995062b77b7d,3.10.5,NaN,NaN
71,2020-02-18 00:07:55,orderTimerRendered,5df100cb1471f12b2a90694c,5e4a755d9a773437e4b65755,3.10.5,NaN,NaN
...,...,...,...,...,...,...,...
1410239,2020-02-18 23:58:56,CaptainOrderStatus-riderBusy,5d5ba8a4bde10e0d5449403e,5e4c2ccb865d9017b540f5ac,3.10.5,NaN,True
1410310,2020-02-18 23:58:45,orderTimerRendered,5dca8c270856f1461825a6e2,5e4c2cd49a773437e4bab532,3.10.5,NaN,NaN
1410343,2020-02-18 23:58:58,orderTimerRendered,5bb4b327d2f6245a30f91370,5e4c2cd69a773437e4bab535,3.10.5,NaN,NaN
1410523,2020-02-18 23:59:53,orderTimerRendered,"58681b57d3ba27eb3228dd46,5e03c4c518576832ff7cf...",5e4c2d009a773437e4bab553,3.10.5,NaN,NaN


In [328]:
# result.mask()

In [145]:
test_frame=sorted_events.head(5000)
# sorted_events_reset[sorted_events_reset['eventName']=='CaptainOrderStatus-riderBusy']

In [151]:
test_frame.reset_index(drop=True,inplace=True)

In [152]:
def get_rider_busy_and_rendered(x):
    return x[x['eventName'].isin(['orderTimerRendered','CaptainOrderStatus-riderBusy'])]

In [154]:
render_and_busy=test_frame.groupby(['order_id','captain_id']).apply(get_rider_busy_and_rendered)

In [160]:
render_and_busy.head(40)
r_b_2=render_and_busy.reset_index(drop=True)

,time,eventName,captain_id,order_id,app_version,cancel_reason
0,2020-02-18 18:50:10,orderTimerRendered,5d6726d05176ff1173707034,5e39137351b5a35da32437c3,3.10.4,NaN
1,2020-02-18 16:13:31,orderTimerRendered,5bd7c083a0b5cd48cd97845a,5e3aad806c2b60097fd384d0,3.10.5,NaN
2,2020-02-18 23:32:16,orderTimerRendered,"5c7416d95e042733c9be335d,5c70948ede141933060f9acb",5e45ab09865d9017b5328bb2,3.10.5,NaN
3,2020-02-18 18:19:04,orderTimerRendered,5cd04a003a47bc2391fd3b10,5e46880c9a773437e4ae1a8d,3.10.5,NaN
4,2020-02-18 18:53:18,orderTimerRendered,5d3c064bbc80ee0ae84c1763,5e476db62806995062b24615,3.10.5,NaN
5,2020-02-18 23:57:35,orderTimerRendered,5c444b954a267149c7754c8b,5e487bc9865d9017b538cb9d,3.10.5,NaN
6,2020-02-18 18:46:54,orderTimerRendered,"5c7115bc5e042733c9bca5fe,5d1d7cbe11e17032dceaa3b7",5e48e49e865d9017b5394b34,3.10.5,NaN
7,2020-02-18 23:54:29,orderTimerRendered,5d1d96e493686945e711bb94,5e48f9a72806995062b581e9,3.10.5,NaN
8,2020-02-18 23:54:33,orderTimerRendered,5d1d96e493686945e711bb94,5e48f9a72806995062b581e9,3.10.5,NaN
9,2020-02-18 23:55:40,orderTimerRendered,5d1d96e493686945e711bb94,5e48f9a72806995062b581e9,3.10.5,NaN


In [187]:
test=r_b_2.iloc[17:32]

In [224]:
def process_test(test):
    test.reset_index(drop=True,inplace=True)

    rider_busy_indices=test[test['eventName']=='CaptainOrderStatus-riderBusy'].index
    def prev_event_ordertimer(test,prev_index):
        try:
            return test.iloc[prev_index]['eventName']=='orderTimerRendered'
        except:
            return False
    result=[(prev_event_ordertimer(test,prev_index)) for prev_index in  rider_busy_indices-1]
    l=list(zip(result,rider_busy_indices))
    l
    r2=list(filter(lambda x: x[0]==True, l))
    succes_index=[x[1] for x in r2]
    return test.iloc[succes_index]

In [231]:
test=r_b_2.iloc[17:32]
process_test(test)
test=r_b_2.iloc[12:18]
process_test(test)

,time,eventName,captain_id,order_id,app_version,cancel_reason
6,2020-02-18 05:29:55,CaptainOrderStatus-riderBusy,5d12261f4253186bbaff6174,5e49a3159a773437e4b48189,3.10.2,NaN
9,2020-02-18 05:30:25,CaptainOrderStatus-riderBusy,5d12261f4253186bbaff6174,5e49a3159a773437e4b48189,3.10.2,NaN


,time,eventName,captain_id,order_id,app_version,cancel_reason
2,2020-02-18 01:27:04,CaptainOrderStatus-riderBusy,5cbee92b3d65ca5e2560ac41,5e499c1d865d9017b53ac1ae,3.10.5,NaN
4,2020-02-18 01:27:32,CaptainOrderStatus-riderBusy,5cbee92b3d65ca5e2560ac41,5e499c1d865d9017b53ac1ae,3.10.5,NaN


In [230]:
r_b_2.iloc[12:18]

,time,eventName,captain_id,order_id,app_version,cancel_reason
12,2020-02-18 23:08:35,orderTimerRendered,5dac1d8b341b00418cf15672,5e4987f19a773437e4b472c2,3.10.5,NaN
13,2020-02-18 01:26:39,orderTimerRendered,5b7bdcfb841e3005f30e267b,5e499c1d865d9017b53ac1ae,3.10.5,NaN
14,2020-02-18 01:27:04,CaptainOrderStatus-riderBusy,5cbee92b3d65ca5e2560ac41,5e499c1d865d9017b53ac1ae,3.10.5,NaN
15,2020-02-18 01:27:08,orderTimerRendered,5cbee92b3d65ca5e2560ac41,5e499c1d865d9017b53ac1ae,3.10.5,NaN
16,2020-02-18 01:27:32,CaptainOrderStatus-riderBusy,5cbee92b3d65ca5e2560ac41,5e499c1d865d9017b53ac1ae,3.10.5,NaN
17,2020-02-18 05:30:26,orderTimerRendered,5b83e8f28be5c25c97eed860,5e49a3159a773437e4b48189,3.10.5,NaN


In [297]:
r_b_2.head(2)

,time,eventName,captain_id,order_id,app_version,cancel_reason
0,2020-02-18 18:50:10,orderTimerRendered,5d6726d05176ff1173707034,5e39137351b5a35da32437c3,3.10.4,NaN
1,2020-02-18 16:13:31,orderTimerRendered,5bd7c083a0b5cd48cd97845a,5e3aad806c2b60097fd384d0,3.10.5,NaN


In [182]:
def run(x):
    return get_rider_busy_indices(x)
def get_rider_busy_indices(x):
    rider_busy_indices=x[x['eventName']=='CaptainOrderStatus-riderBusy'].index
    if len(rider_busy_indices) > 0:
        return rider_busy_indices
    return None
# def operate(x):
    
#     prev_indices=rider_busy_indices-1
#     result=[]
#     for each_prev_index in prev_indices:
#         if each_prev_index in x.index:
#             success=x.iloc[each_prev_index]['event_Name']=='orderTimerRendered'
#         else:
#             success=False
#         result.append([each_prev_index,success])
#     return result

In [184]:
r_b_2.groupby(['order_id','captain_id']).apply(get_rider_busy_indices)

TypeError: Must pass either `items`, `like`, or `regex`

In [ ]:
df.groupby(['job']).apply(lambda x: (x.groupby('source')
                                      .sum()
                                      .sort_values('count', ascending=False))
                                     .head(3))